# Matching algorithm via keywords

### Prerequisites

Download processed datasets via `git lfs fetch --include "data/processed/*.csv"`

or create them from scratch with [preprocess.ipynb](./preprocess.ipynb)

In [1]:
!pip install pandas
!pip install beautifulsoup4
!pip install scikit-learn
!pip install nltk
!pip install sentence-transformers
!pip install numpy

!pip install rake-nltk
!pip install pip install git+https://github.com/LIAAD/yake
!pip install keybert

  Cloning https://github.com/LIAAD/yake to /private/var/folders/3y/lpnlj_f962vbckl7626fst540000gq/T/pip-req-build-wtdsmuoj
  Running command git clone --filter=blob:none --quiet https://github.com/LIAAD/yake /private/var/folders/3y/lpnlj_f962vbckl7626fst540000gq/T/pip-req-build-wtdsmuoj
  Resolved https://github.com/LIAAD/yake to commit 374fc1c1c19eb080d5b6115cbb8d4a4324392e54
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import os
import os.path as path
import pandas as pd
from bs4 import BeautifulSoup
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np

from rake_nltk import Rake
import yake
from keybert import KeyBERT

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alv.popov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/alv.popov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
root_dir = path.dirname(path.abspath(os.getcwd()))
data_dir = path.join(root_dir, 'data')
raw_data_dir = path.join(data_dir, 'raw')

raw_vacancies_path = path.join(raw_data_dir, 'hhparser_vacancy.csv')
raw_cvs_path = path.join(raw_data_dir, 'dst-3.0_16_1_hh_database.csv')

raw_vacancies_path

'/Users/alv.popov/prj/DL-CASE2/data/raw/hhparser_vacancy.csv'

In [4]:
def process_description(description):
    description = str(description)
    description = BeautifulSoup(description, features="html.parser").get_text()
    return description

raw_vacancies = pd.read_csv(raw_vacancies_path, index_col='id')
raw_vacancies['text'] = (
    raw_vacancies['name'] + " . " +
    raw_vacancies['description'].apply(process_description) + " . " +
    raw_vacancies['area_name']
).astype(str)
raw_vacancies = raw_vacancies[raw_vacancies['text'].notnull()]
raw_vacancies

/var/folders/3y/lpnlj_f962vbckl7626fst540000gq/T/ipykernel_73711/2199012901.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  description = BeautifulSoup(description, features="html.parser").get_text()


,name,has_test,response_letter_required,salary_from,salary_to,salary_currency,salary_gross,published_at,created_at,parsed_at,url,employer_name,description,alternate_url,area_id,area_name,text
id,,,,,,,,,,,,,,,,,
38228194,Project R’n’D / Технолог R’n’D,False,True,NaN,NaN,NaN,NaN,2020-07-26 23:40:42+00,2020-07-26 23:40:42+00,2020-07-27 03:03:03.154874+00,https://api.hh.ru/vacancies/38228194?host=hh.ru,Масл фектори,<p><strong>MF Kitchen</strong> — группа компан...,https://hh.ru/vacancy/38228194,1,Москва,Project R’n’D / Технолог R’n’D . MF Kitchen — ...
36911636,"Продавец-кассир (Москва, Богданова, 16)",False,False,33500.0,40000.0,RUR,True,2020-07-27 00:28:12+00,2020-07-27 00:28:12+00,2020-07-27 03:03:04.258759+00,https://api.hh.ru/vacancies/36911636?host=hh.ru,Пятёрочка,<p><strong>Обязанности:</strong></p><ul><li>Об...,https://hh.ru/vacancy/36911636,1,Москва,"Продавец-кассир (Москва, Богданова, 16) . Обяз..."
36909000,"Продавец-кассир (Москва, Новогиреевская, 11/36)",False,False,33500.0,38000.0,RUR,True,2020-08-24 00:16:48+00,2020-08-24 00:16:48+00,2020-08-24 02:59:43.766137+00,https://api.hh.ru/vacancies/36909000?host=hh.ru,Пятёрочка,<p><strong>Обязанности:</strong></p><ul><li>Об...,https://hh.ru/vacancy/36909000,1,Москва,"Продавец-кассир (Москва, Новогиреевская, 11/36..."
38985954,"Product owner бизнес–стрима ""Аналитика и Отчет...",False,False,NaN,NaN,NaN,NaN,2020-09-08 18:29:25+00,2020-09-08 18:29:25+00,2020-09-08 19:02:47.261747+00,https://api.hh.ru/vacancies/38985954?host=hh.ru,Банк ВТБ (ПАО),<p>Наша позиция предполагает участие в програм...,https://hh.ru/vacancy/38985954,1,Москва,"Product owner бизнес–стрима ""Аналитика и Отчет..."
40108800,Business Development Director (CTO),False,False,NaN,NaN,NaN,NaN,2020-11-25 08:56:45+00,2020-11-25 08:56:45+00,2020-11-28 06:59:37.872153+00,https://api.hh.ru/vacancies/40108800?host=hh.ru,Студия Олега Чулакова,<p>Студия Олега Чулакова — это дизайн-студия №...,https://hh.ru/vacancy/40108800,1,Москва,Business Development Director (CTO) . Студия О...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38834939,Оператор 1C,False,False,48000.0,NaN,RUR,True,2020-08-26 06:52:49+00,2020-08-26 06:52:49+00,2020-08-26 07:02:58.340756+00,https://api.hh.ru/vacancies/38834939?host=hh.ru,ТД ЛФБ,"<p> </p> <p>ТД «ЛФБ» - семейная компания, веде...",https://hh.ru/vacancy/38834939,1,Москва,"Оператор 1C . ТД «ЛФБ» - семейная компания, ..."
37345373,Стажер Управления финансирования торговли,False,False,40000.0,40000.0,RUR,True,2020-07-25 13:23:01+00,2020-07-25 13:23:01+00,2020-07-28 11:01:18.939989+00,https://api.hh.ru/vacancies/37345373?host=hh.ru,UniCredit Bank,<p>КРАТКАЯ ИНФОРМАЦИЯ:</p> <p>Отдел финансиров...,https://hh.ru/vacancy/37345373,1,Москва,Стажер Управления финансирования торговли . КР...
38602820,Преподаватель математики/информатики,False,False,NaN,NaN,NaN,NaN,2020-08-13 16:30:47+00,2020-08-13 16:30:47+00,2020-08-15 07:01:49.548725+00,https://api.hh.ru/vacancies/38602820?host=hh.ru,Центр дополнительного образования Новый Взгляд,<p><strong>Обязанности:</strong></p> <p> </p> ...,https://hh.ru/vacancy/38602820,1,Москва,Преподаватель математики/информатики . Обязанн...


In [5]:
raw_cvs = pd.read_csv(raw_cvs_path, index_col='ind')
raw_cvs['text'] = (
    raw_cvs['Пол, возраст'] + " . " +
    raw_cvs['Ищет работу на должность:'] + " . " +
    raw_cvs['Город, переезд, командировки'] + " . " +
    raw_cvs['Занятость'] + " . " +
    raw_cvs['График'] + " . " +
    raw_cvs['Опыт работы'] + " . " +
    raw_cvs['Последнее/нынешнее место работы'] + " . " +
    raw_cvs['Последняя/нынешняя должность'] + " . " +
    raw_cvs['Образование и ВУЗ']
).astype(str)
raw_cvs = raw_cvs[raw_cvs['text'].notnull()]
raw_cvs

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,text
ind,,,,,,,,,,,,,
0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль,"Мужчина , 39 лет , родился 27 ноября 1979 . С..."
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано,"Мужчина , 60 лет , родился 20 марта 1959 . Те..."
2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано,"Женщина , 36 лет , родилась 12 августа 1982 ...."
3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано,"Мужчина , 38 лет , родился 25 июня 1980 . Веб..."
4,"Женщина , 26 лет , родилась 3 марта 1993",140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Опыт работы 5 лет 7 месяцев Региональный мене...,Мармелад,Менеджер по продажам,Высшее образование 2015 Кгу Психологии и педаг...,22.04.2019 10:32,Не указано,"Женщина , 26 лет , родилась 3 марта 1993 . Ре..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44739,"Мужчина , 30 лет , родился 17 января 1989",50000 руб.,"Финансист, аналитик, экономист, бухгалтер, мен...","Тверь , готов к переезду (Москва, Химки) , гот...",полная занятость,"полный день, удаленная работа","Опыт работы 7 лет 7 месяцев Финансист, аналит...","ООО ""IAS"" (независимый участник объединения Ru...",Руководитель субгруппы,Высшее образование 2015 Московский гуманитарн...,22.04.2019 12:32,Не указано,"Мужчина , 30 лет , родился 17 января 1989 . Ф..."
44740,"Мужчина , 27 лет , родился 5 марта 1992",39000 руб.,"Системный администратор, IT-специалист","Липецк , готов к переезду , готов к командировкам","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день, ...","Опыт работы 7 лет Системный администратор, IT...",ИП Пестрецов,Предприниматель,Высшее образование (Бакалавр) 2016 Воронежски...,22.04.2019 13:11,Не указано,"Мужчина , 27 лет , родился 5 марта 1992 . Сис..."
44741,"Женщина , 48 лет , родилась 26 декабря 1970",40000 руб.,"Аналитик данных, Математик","Челябинск , готова к переезду , готова к редки...",полная занятость,"полный день, удаленная работа",Опыт работы 21 год 5 месяцев Январь 1998 — по...,"ОАО «ЧМК», Исследовательско-Технологический Це...",Начальник группы аналитики,Высшее образование 2000 Южно-Уральский госуда...,09.04.2019 05:07,Не указано,"Женщина , 48 лет , родилась 26 декабря 1970 ...."


In [6]:
# def extract_keywords(text, max_length=5, n_keywords=10):
#     rake = Rake(language='russian', max_length=max_length)
#     rake.extract_keywords_from_text(str(text))
#     return rake.get_ranked_phrases()[:n_keywords]

rake = Rake(language='russian')

def extract_keywords(text, max_length=5, n_keywords=10):
    extractor = yake.KeywordExtractor(
        stopwords=rake.to_ignore,
        windowsSize=max_length,
        top=n_keywords
    )
    scored_keywords = extractor.extract_keywords(str(text))
    return list(map(lambda x: x[0], scored_keywords))


# !!!! KeyBert не разбивает на токены,  !!!!

# extractor = KeyBERT('cointegrated/LaBSE-en-ru')

# def extract_keywords(text, max_length=5, n_keywords=10):
#     scored_keywords = extractor.extract_keywords(
#         str(text),
#         keyphrase_ngram_range=(1, max_length),
#         stop_words=rake.to_ignore,
#         top_n=n_keywords,
#     )
#     return list(map(lambda x: x[0], scored_keywords))

In [7]:
vacancy = raw_vacancies.iloc[121]['text']
vacancy_keywords = extract_keywords(vacancy, max_length=1, n_keywords=30)

display(vacancy)
display(vacancy_keywords)

'C++ разработчик в Германию . Мы ищем C++ разработчиков разных уровней от junior до senior с переездом в Берлин.  Наш клиент - немецкая компания, 17 лет на рынке. Их разработка - программное обеспечение для создания диаграмм и макетов, распознавания их с бумажных носителей. Продукт интегрируется в PowerPoint и автоматизирует его работу за счет применения сложных алгоритмов. Повышает эффективность и качество создания слайдов, в несколько раз экономит время пользователю при подготовке презентации. Поставляется с внутренней таблицей данных на основе Excel и интеллектуальными ссылками. Продукт работает и по Windows, и под Mac. При его выходе на рынок это была настоящая революция. Компания занимала первые места в рейтингах лучших стартапов Германии и быстрорастущих бизнесов. У компании 700 000 клиентов по всему миру. Среди них, например, Deloitte, Ernst & Young, Coca-Cola, DHL, 3M, Cisco. В чем еще участвует компания:  спонсор Standard C++ Foundation - ведущего мирового некоммерческого сооб

['разработчик в Германию',
 'разработчиков разных уровней',
 'разных уровней',
 'уровней от junior',
 'junior до senior',
 'senior с переездом',
 'переездом в Берлин',
 'разработчиков разных',
 'Германию',
 'Наш клиент',
 'лет на рынке',
 'разработчик',
 'Продукт',
 'английском языке',
 'ищем',
 'разных',
 'уровней',
 'junior',
 'senior',
 'переездом',
 'компания',
 'Берлин',
 'немецкая компания',
 'качество создания слайдов',
 'Германии',
 'CTO',
 'Джоном Форрестом',
 'создания диаграмм',
 'создания',
 'применения сложных алгоритмов']

In [8]:
cv = raw_cvs.iloc[815]['text']
cv_keywords = extract_keywords(cv, n_keywords=30)

display(cv)
display(cv_keywords)

'Мужчина ,  31 год , родился 8 сентября 1987 . Специалист по IT . Москва ,  м. Жулебино , готов к переезду , готов к командировкам . полная занятость . полный день . Опыт работы 8 лет 10 месяцев  Сентябрь 2018 — по настоящее время 9 месяцев Print grad Инженер-программист Пуско наладочные работы, монтаж Скс, видео  Октябрь 2010 — Октябрь  2016 6 лет 1 месяц Sorex Москва Информационные технологии, системная интеграция, интернет ... Системная интеграция, автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг Инженер-программист Слаботочные системы , пуско наладочные работы , объекты под ключ  Октябрь 2013 — Октябрь  2014 1 год 1 месяц Касспромсервис Москва Информационные технологии, системная интеграция, интернет ... Системная интеграция, автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг Услуги для бизнеса Безопасность, охранная деятельность Инженер Объекты под ключь, сборка видео и монтаж Магазинов сети x5 retail  Октябрь 2010 — Октябрь  201

['Москва Информационные технологии',
 'системная интеграция',
 'месяц Sorex Москва',
 'Sorex Москва Информационные',
 'Касспромсервис Москва Информационные',
 'Москва Информационные',
 'Shark Москва Информационные',
 'Октябрь',
 'Информационные технологии',
 'месяц Касспромсервис Москва',
 'Sorex Москва Монтажник',
 'месяц Shark Москва',
 'Sorex Москва',
 'Москва',
 'Пуско наладочные работы',
 'системная',
 'интеграция',
 'Инженер-программист Пуско наладочные',
 'ИТ-консалтинг Инженер-программист Слаботочные',
 'Москва Личный водитель',
 'монтаж видео Октябрь',
 'видео Октябрь',
 'автоматизации технологических',
 'бизнес-процессов предприятия',
 'Print grad Инженер-программист',
 'grad Инженер-программист Пуско',
 'технологических и бизнес-процессов',
 'месяц Sorex',
 'Инженер-программист Слаботочные системы',
 'фриланс Москва Личный']

In [9]:
model = SentenceTransformer('cointegrated/LaBSE-en-ru')

res = cosine_similarity(model.encode(vacancy_keywords), model.encode(cv_keywords)).ravel()
print(
f"""
Min: {res.min()}
Mean: {res.mean()}
Max: {res.max()}
Harmonic: {len(res) / sum(1 / res)}
Median: {np.median(res)}
"""
)


Min: -0.0604478158056736
Mean: 0.22960664331912994
Max: 0.5930092334747314
Harmonic: 0.17163926307209573
Median: 0.21927300095558167



In [10]:
def calc_metric(v1, v2):
    return np.median(cosine_similarity(v1, v2))

calc_metric(model.encode(vacancy_keywords), model.encode(cv_keywords))

0.219273

In [12]:
n_items = 100
n_keywords = 10
max_length = 5

def encode_batch(keywords_batch):
    encoded = model.encode(np.concatenate(keywords_batch).tolist())
    lengths = np.fromiter(map(len, keywords_batch), dtype=int)
    encoded_keywords_iter = (encoded[end - l: end] for end, l in zip(np.cumsum(lengths), lengths))
    return np.fromiter(encoded_keywords_iter, dtype=np.ndarray)

def get_encodings(dataset):
    dataset = dataset.apply(lambda x: extract_keywords(x, max_length, n_keywords))
    dataset = dataset.tolist()
    return encode_batch(dataset)

sampled_vacancies = raw_vacancies.sample(n_items)
sampled_cvs = raw_cvs.sample(n_items)

vacancies_encodings = get_encodings(sampled_vacancies['text'])
cvs_encodings = get_encodings(sampled_cvs['text'])

results = np.array([[calc_metric(cv_encoding, vac_encoding) for cv_encoding in cvs_encodings] for vac_encoding in vacancies_encodings])
print('\n'.join(map(lambda x: '\t'.join(map(str, x)), results[:10, :10])))

0.25691175	0.28795505	0.28818244	0.28562665	0.23925231	0.28093418	0.19965295	0.26002872	0.2510838	0.2842012
0.24706432	0.222266	0.26551408	0.24878897	0.24272871	0.2538979	0.27413064	0.27021134	0.18362314	0.2634074
0.26437992	0.22591972	0.24286175	0.1684229	0.24919465	0.22173797	0.16227576	0.22526065	0.19179702	0.21692242
0.30028662	0.34675768	0.2478281	0.23594162	0.25927034	0.22417888	0.30499876	0.26363927	0.23364428	0.2522561
0.2427453	0.25031173	0.23366553	0.19852966	0.26317945	0.21131912	0.1916992	0.25403345	0.20527555	0.20554262
0.21817423	0.19821085	0.26624036	0.18087429	0.23731557	0.2544093	0.20758438	0.24951962	0.18710509	0.21686336
0.24797235	0.29973352	0.2872331	0.22289968	0.25001654	0.2679367	0.22345263	0.2718841	0.22176135	0.24731281
0.21563023	0.25901547	0.24723426	0.17896932	0.23307154	0.2376513	0.18754488	0.24644706	0.18258847	0.21188116
0.18339932	0.19533156	0.22670363	0.21555634	0.20383084	0.22790965	0.13923214	0.22219086	0.15711768	0.22506976
0.17551476	0.1777165	0.237

In [13]:
np.where(results > 0.45)

(array([], dtype=int64), array([], dtype=int64))

In [14]:
kw1 = extract_keywords(sampled_vacancies.iloc[6]['text'], max_length, n_keywords)
kw2 = extract_keywords(sampled_cvs.iloc[5]['text'], max_length, n_keywords)

metric = calc_metric(model.encode(kw1), model.encode(kw2))
display(kw1, kw2, metric)

['передачи данных Просьба',
 'данных Просьба откликаться',
 'вдумчивый системный аналитик',
 'который понимает хотя',
 'коллективеЧем предстоит заниматься',
 'полностью удалённая работа',
 'лёгкостью описывает процессы.Что',
 'молодом коллективеЧем предстоит',
 'внедрять варианты схем',
 'варианты схем работы']

['vernonwear.com Розничная торговля',
 'Опыт работы',
 'готов к переезду',
 'готов к командировкам',
 'месяца Апрель',
 'полная занятость',
 'полный день',
 'Интернет-маркетолог',
 'vernonwear.com Розничная',
 'Розничная торговля']

0.2679367